# RNN

## Equipo 4:


*   Karla Andrea Palma Villanueva (A01754270)
*   Viviana Alanis Fraige (A01236316)
* David Fernando Armendariz Torres (A01570813)
* Alan Alberto Mota Yescas (A01753924)
* Adrián Chávez Morales (A01568679)
* Jose Manuel Armendáriz Mena (A01197583)

### Introducción


En este notebook, implementamos un modelo de Deep Learning para clasificar actividades humanas utilizando el UCI HAR Dataset. Este conjunto de datos contiene lecturas de sensores de un teléfono inteligente usado por personas mientras realizan diferentes actividades. Nuestro objetivo es construir un modelo que clasifique correctamente cada actividad a partir de las características obtenidas del sensor.


### Exploración, explicación y limpieza de datos

#### Origen y Contexto del Dataset UCI-HAR

El UCI HAR Dataset (Human Activity Recognition Using Smartphones) proviene del Machine Learning Repository de la Universidad de California, Irvine (UCI). Este dataset fue creado para ayudar en el desarrollo de modelos de reconocimiento de actividades humanas utilizando datos de sensores de smartphones.

#### Descripción del Dataset
El objetivo del dataset es clasificar distintas actividades humanas en función de datos recogidos por un teléfono inteligente. Los datos fueron recopilados por el acelerómetro y giroscopio del smartphone mientras 30 voluntarios realizaban diferentes actividades en un entorno controlado. Las actividades incluyen: caminar, subir escaleras, bajar escaleras, sentarse, pararse y recostarse.

Cada participante llevó un teléfono inteligente con sensores inerciales en la cintura, y los datos se registraron a una frecuencia constante para capturar mediciones en los ejes X, Y y Z. Esto permite capturar las características de cada actividad de manera precisa, lo cual es crucial para la tarea de clasificación.

#### Obtención del Dataset
Este conjunto de datos está disponible públicamente en el UCI Machine Learning Repository. Se usa ampliamente en investigación y en la industria para desarrollar y mejorar modelos de reconocimiento de actividades humanas.

Link: [https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones](https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones).



In [ ]:
#Importación de librerías
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM, GRU
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
from keras_tuner import Hyperband
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve


In [ ]:

# Cargar los nombres de las características
features = pd.read_csv('C:/Users/Alan/Documents/Alan/TEC/TAREAS/Semestre TEC 7mo/IA avanzada/Periodo 2 y 3/Deep Learning/UCI-HAR Dataset/features.txt', delim_whitespace=True, header=None)
feature_names = features[1].values

# Verificar duplicados en feature_names
if pd.Index(feature_names).duplicated().any():
    feature_names = pd.Series(feature_names).where(~pd.Series(feature_names).duplicated(), other=lambda x: x + "_" + x.index.astype(str)).tolist()

# Cargar las etiquetas de actividad
activity_labels = pd.read_csv('C:/Users/Alan/Documents/Alan/TEC/TAREAS/Semestre TEC 7mo/IA avanzada/Periodo 2 y 3/Deep Learning/UCI-HAR Dataset/activity_labels.txt', delim_whitespace=True, header=None, index_col=0)

# Cargar datos de entrenamiento
X_train = pd.read_csv('C:/Users/Alan/Documents/Alan/TEC/TAREAS/Semestre TEC 7mo/IA avanzada/Periodo 2 y 3/Deep Learning/UCI-HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=feature_names)
y_train = pd.read_csv('C:/Users/Alan/Documents/Alan/TEC/TAREAS/Semestre TEC 7mo/IA avanzada/Periodo 2 y 3/Deep Learning/UCI-HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None, names=['Activity'])

# Cargar datos de prueba
X_test = pd.read_csv('C:/Users/Alan/Documents/Alan/TEC/TAREAS/Semestre TEC 7mo/IA avanzada/Periodo 2 y 3/Deep Learning/UCI-HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=feature_names)
y_test = pd.read_csv('C:/Users/Alan/Documents/Alan/TEC/TAREAS/Semestre TEC 7mo/IA avanzada/Periodo 2 y 3/Deep Learning/UCI-HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None, names=['Activity'])






C:\Users\Alan\AppData\Local\Temp\ipykernel_14868\2723011189.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  features = pd.read_csv('C:/Users/Alan/Documents/Alan/TEC/TAREAS/Semestre TEC 7mo/IA avanzada/Periodo 2 y 3/Deep Learning/UCI-HAR Dataset/features.txt', delim_whitespace=True, header=None)
C:\Users\Alan\AppData\Local\Temp\ipykernel_14868\2723011189.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  activity_labels = pd.read_csv('C:/Users/Alan/Documents/Alan/TEC/TAREAS/Semestre TEC 7mo/IA avanzada/Periodo 2 y 3/Deep Learning/UCI-HAR Dataset/activity_labels.txt', delim_whitespace=True, header=None, index_col=0)
C:\Users\Alan\AppData\Local\Temp\ipykernel_14868\2723011189.py:15: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future ve

#### Análisis del dataset: número de columnas, instancias, tipos de datos.


In [ ]:
# dimensiones de los datos
print("Dimensiones de los datos de entrenamiento (X_train):", X_train.shape)
print("Dimensiones de las etiquetas de entrenamiento (y_train):", y_train.shape)
print("Dimensiones de los datos de prueba (X_test):", X_test.shape)
print("Dimensiones de las etiquetas de prueba (y_test):", y_test.shape)

# tipos de datos
print("\nTipos de datos en X_train:")
print(X_train.dtypes)
print("\nTipos de datos en y_train:")
print(y_train.dtypes)
print("\nTipos de datos en X_test:")
print(X_test.dtypes)
print("\nTipos de datos en y_test:")
print(y_test.dtypes)


Dimensiones de los datos de entrenamiento (X_train): (7352, 561)
Dimensiones de las etiquetas de entrenamiento (y_train): (7352, 1)
Dimensiones de los datos de prueba (X_test): (2947, 561)
Dimensiones de las etiquetas de prueba (y_test): (2947, 1)

Tipos de datos en X_train:
tBodyAcc-mean()-X                       float64
tBodyAcc-mean()-Y                       float64
tBodyAcc-mean()-Z                       float64
tBodyAcc-std()-X                        float64
tBodyAcc-std()-Y                        float64
                                         ...   
angle(tBodyGyroMean,gravityMean)        float64
angle(tBodyGyroJerkMean,gravityMean)    float64
angle(X,gravityMean)                    float64
angle(Y,gravityMean)                    float64
angle(Z,gravityMean)                    float64
Length: 561, dtype: object

Tipos de datos en y_train:
Activity    object
dtype: object

Tipos de datos en X_test:
tBodyAcc-mean()-X                       float64
tBodyAcc-mean()-Y                

Los datos de entrenamiento (`X_train`) contienen 7352 instancias, cada una con 561 columnas que representan diferentes características capturadas de los sensores inerciales del smartphone, como aceleración y giroscopio. Estas características incluyen tanto valores de medias y desviaciones estándar en los tres ejes (X, Y, Z) como ángulos entre vectores de aceleración y gravedad. Todos los datos en `X_train` están en formato `float64`, lo cual asegura precisión en los cálculos. 

En cuanto a las etiquetas de actividad (`y_train`), están representadas en una columna bajo el nombre "Activity" y están en formato `object`, ya que contienen nombres categóricos que representan cada tipo de actividad, como caminar, subir o bajar escaleras. Los datos de prueba (`X_test` y `y_test`) siguen la misma estructura, pero con 2947 instancias en lugar de 7352. La consistencia en la estructura de entrenamiento y prueba facilita el uso de estos datos en modelos de clasificación, ya que las mismas características y etiquetas están presentes en ambos conjuntos, lo cual es crucial para el entrenamiento y la validación del modelo de reconocimiento de actividades humanas.

In [ ]:

# Distribución de actividades en el conjunto de entrenamiento
train_distribution = y_train['Activity'].value_counts().reset_index()
train_distribution.columns = ['Activity', 'Count']

# Distribución de actividades en el conjunto de prueba
test_distribution = y_test['Activity'].value_counts().reset_index()
test_distribution.columns = ['Activity', 'Count']

# Gráfico de entrenamiento
fig_train = px.bar(train_distribution, x='Activity', y='Count', 
                   title='Distribución de actividades - Entrenamiento',
                   labels={'Activity': 'Actividad', 'Count': 'Cantidad de muestras'},
                   color='Activity')
fig_train.show()

# Gráfico de prueba
fig_test = px.bar(test_distribution, x='Activity', y='Count', 
                  title='Distribución de actividades - Prueba',
                  labels={'Activity': 'Actividad', 'Count': 'Cantidad de muestras'},
                  color='Activity')
fig_test.show()


c:\Users\Alan\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1979: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


c:\Users\Alan\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Las gráficas muestran la distribución de muestras para cada actividad en los conjuntos de datos de entrenamiento y prueba. Cada barra representa una actividad específica (como "LAYING", "STANDING", "WALKING", etc.), y su altura indica la cantidad de muestras correspondientes a esa actividad en el conjunto de datos.

En el conjunto de entrenamiento, observamos que las actividades están bastante equilibradas, con un número similar de muestras en cada categoría. Esto es ideal para el entrenamiento de modelos de clasificación, ya que permite que el modelo aprenda de manera uniforme las características de cada actividad sin sesgo hacia alguna clase específica.

La distribución en el conjunto de prueba también es equilibrada y refleja la distribución del conjunto de entrenamiento, lo cual es importante para evaluar de forma justa el rendimiento del modelo en todas las actividades. La consistencia en la cantidad de muestras entre los conjuntos de entrenamiento y prueba facilita una evaluación robusta y evita el sesgo hacia actividades con mayor o menor cantidad de datos.


#### Proceso de limpieza y verificación para el modelo

Verificación de datos

In [14]:
# Verificar valores faltantes en X_train y X_test
print("Valores faltantes en X_train:", X_train.isnull().sum().sum())
print("Valores faltantes en X_test:", X_test.isnull().sum().sum())

# Verificar valores faltantes en y_train y y_test
print("Valores faltantes en y_train:", y_train.isnull().sum().sum())
print("Valores faltantes en y_test:", y_test.isnull().sum().sum())


Valores faltantes en X_train: 0
Valores faltantes en X_test: 0
Valores faltantes en y_train: 0
Valores faltantes en y_test: 0


In [16]:
# Verificar duplicados en X_train y X_test
print("Duplicados en X_train:", X_train.duplicated().sum())
print("Duplicados en X_test:", X_test.duplicated().sum())

# Verificar duplicados en y_train y y_test
print("Duplicados en y_train:", y_train.duplicated().sum())
print("Duplicados en y_test:", y_test.duplicated().sum())


Duplicados en X_train: 0
Duplicados en X_test: 0
Duplicados en y_train: 7346
Duplicados en y_test: 2941


En el proceso de limpieza y verificación de datos, primero verificamos la existencia de valores faltantes en ambos conjuntos de datos (`X_train`, `X_test`, `y_train`, `y_test`). Los resultados muestran que no hay valores faltantes en ninguno de los conjuntos, lo cual es un indicio positivo, ya que significa que no es necesario realizar un proceso de imputación de datos ni tomar otras medidas para lidiar con valores nulos. Esto simplifica el preprocesamiento, permitiéndonos proceder directamente al desarrollo del modelo sin perder información debido a la ausencia de datos.

Además, verificamos la existencia de datos duplicados. En el caso de `X_train` y `X_test`, no se encontraron duplicados, lo cual asegura que cada muestra en estos conjuntos representa una observación única, lo que es esencial para un entrenamiento robusto del modelo. Sin embargo, en `y_train` y `y_test`, se observó un número considerable de duplicados (7346 y 2941, respectivamente). Esto es esperable y aceptable en el contexto de clasificación de actividades humanas, ya que las etiquetas representan categorías de actividades repetitivas. Este tipo de duplicados en las etiquetas no afecta negativamente al modelo, ya que el objetivo es identificar patrones en los datos que correspondan a las distintas actividades.


#### Desarrollo del Modelo de Deep Learning

En la sección de desarrollo del modelo de Deep Learning, es crucial realizar los pasos de **preparación de datos** y **construcción de la arquitectura** del modelo para garantizar que el modelo pueda aprender de manera efectiva a partir de los datos de entrada.

Primero, en el código se realiza la **conversión de etiquetas a formato categórico** utilizando la función `to_categorical`. Dado que estamos tratando con un problema de clasificación multiclase (clasificación de actividades), las etiquetas deben estar en un formato que el modelo pueda interpretar. Convertir las etiquetas en categorías ayuda a que cada actividad se represente mediante un vector de una sola dimensión con valores binarios (one-hot encoding). Esto permite que el modelo entienda cada clase de manera independiente y mejore la precisión en la predicción de actividades específicas.

El siguiente paso es el **reformateo de los datos de entrada (`X_train` y `X_test`)** para darles la forma requerida por la capa LSTM del modelo. La red LSTM espera que los datos tengan la estructura `[samples, timesteps, features]`, donde:
- **samples**: representa el número total de ejemplos en el conjunto de datos.
- **timesteps**: se refiere a la cantidad de pasos de tiempo, en este caso configurado en `1`, ya que cada instancia representa un solo conjunto de lecturas de sensores.
- **features**: es el número de características (561) extraídas de los sensores.

Este reformateo permite que el modelo LSTM procese cada muestra como una secuencia de características, que es esencial para capturar patrones temporales o secuenciales en los datos de sensores.

Finalmente, se construye la **arquitectura del modelo**. El modelo se inicia con una capa LSTM de 64 unidades, que es ideal para capturar patrones complejos en los datos secuenciales. La capa LSTM es seguida de una capa de `Dropout` (con una tasa de 0.5), que ayuda a prevenir el sobreajuste al desactivar aleatoriamente algunas neuronas durante el entrenamiento, lo cual mejora la generalización del modelo. Luego, una capa `Dense` con 32 neuronas y activación `relu` se utiliza para procesar las características extraídas. La última capa es una capa `Dense` con una activación `softmax`, que produce una probabilidad para cada clase y permite que el modelo determine la actividad específica.

Este diseño del modelo permite capturar relaciones temporales en los datos y tomar decisiones de clasificación precisas en función de las características de los sensores.

In [19]:
# Convertir las etiquetas a formato categórico
y_train_categorical = to_categorical(y_train['Activity'].factorize()[0])
y_test_categorical = to_categorical(y_test['Activity'].factorize()[0])

# Reformatear X_train y X_test para que tengan la forma [samples, timesteps, features]
X_train_reshaped = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))


In [ ]:
# arquitectura del modelo 
model = Sequential()
model.add(LSTM(64, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train_categorical.shape[1], activation='softmax')) 


c:\Users\Alan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [22]:
# Entrenar el modelo
history = model.fit(X_train_reshaped, y_train_categorical, epochs=20, batch_size=32, validation_data=(X_test_reshaped, y_test_categorical))


Epoch 1/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.5919 - loss: 0.9926 - val_accuracy: 0.8670 - val_loss: 0.3065
Epoch 2/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8992 - loss: 0.2628 - val_accuracy: 0.9084 - val_loss: 0.2190
Epoch 3/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9186 - loss: 0.2014 - val_accuracy: 0.9270 - val_loss: 0.1798
Epoch 4/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9453 - loss: 0.1483 - val_accuracy: 0.9260 - val_loss: 0.1865
Epoch 5/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9497 - loss: 0.1259 - val_accuracy: 0.9372 - val_loss: 0.1586
Epoch 6/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9525 - loss: 0.1227 - val_accuracy: 0.9447 - val_loss: 0.1497
Epoch 7/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9558 - loss: 0.1138 - val_accuracy: 0.9444 - val_loss: 0.1602
Epoch 8/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9577 - loss: 0.1132 - val_accuracy: 

In [23]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
print(f"Precisión en el conjunto de prueba: {test_accuracy * 100:.2f}%")


93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9398 - loss: 0.1757
Precisión en el conjunto de prueba: 94.23%


En esta etapa del desarrollo del modelo de Deep Learning, hemos llevado a cabo el proceso de compilación, entrenamiento y evaluación del modelo.

Compilación del Modelo:

El modelo fue compilado utilizando el optimizador Adam, que es una de las opciones más populares y efectivas para el entrenamiento de redes neuronales debido a su capacidad de adaptación en la tasa de aprendizaje durante el proceso de optimización. Se utilizó la función de pérdida categorical_crossentropy, que es adecuada para problemas de clasificación multiclase, ya que mide la diferencia entre la distribución de las clases predichas y las clases reales. También se especificó accuracy como métrica de evaluación, para tener una medida directa de la proporción de predicciones correctas.

Entrenamiento del Modelo:

Durante el entrenamiento, el modelo fue entrenado en 20 épocas con un tamaño de batch de 32. La precisión (`accuracy`) y la pérdida (`loss`) fueron monitoreadas tanto en el conjunto de entrenamiento como en el de validación. A medida que avanzan las épocas, se observa que tanto la precisión de entrenamiento como la de validación mejoran gradualmente, mientras que la pérdida disminuye. Esto indica que el modelo está aprendiendo patrones útiles de los datos sin mostrar indicios de sobreajuste prematuro, ya que las métricas de validación siguen una tendencia similar a las de entrenamiento.

Evaluación en el Conjunto de Prueba:

Después del entrenamiento, el modelo fue evaluado en el conjunto de prueba, y se obtuvo una precisión del 94.23%. Este resultado muestra que el modelo tiene un alto grado de precisión al clasificar actividades en datos no vistos, lo que indica que ha logrado capturar de manera efectiva los patrones de las actividades humanas en los datos de sensores.



In [ ]:

# Gráfico de precisión
fig_accuracy = go.Figure()
fig_accuracy.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines', name='Precisión de Entrenamiento'))
fig_accuracy.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines', name='Precisión de Validación'))
fig_accuracy.update_layout(title='Precisión durante el Entrenamiento',
                           xaxis_title='Época', yaxis_title='Precisión')
fig_accuracy.show()

# Gráfico de pérdida
fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Pérdida de Entrenamiento'))
fig_loss.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='Pérdida de Validación'))
fig_loss.update_layout(title='Pérdida durante el Entrenamiento',
                       xaxis_title='Época', yaxis_title='Pérdida')
fig_loss.show()


En la primera gráfica, que muestra la precisión durante el entrenamiento para los conjuntos de entrenamiento y validación, se observa que la precisión de entrenamiento (línea azul) aumenta rápidamente durante las primeras épocas y luego se estabiliza cerca del 97%. La precisión de validación (línea roja) sigue una tendencia similar, aunque con ligeras fluctuaciones. La proximidad de ambas curvas indica que el modelo generaliza bien y no presenta problemas significativos de sobreajuste, ya que la precisión de validación se mantiene cercana a la precisión de entrenamiento.

La segunda gráfica muestra la pérdida durante el entrenamiento para los conjuntos de entrenamiento y validación. La pérdida de entrenamiento disminuye rápidamente en las primeras épocas, alcanzando valores bajos y estables en épocas posteriores. La pérdida de validación también disminuye de manera consistente y sigue una tendencia similar a la del entrenamiento, aunque con algunas variaciones. La cercanía entre las curvas de pérdida de entrenamiento y validación sugiere que el modelo es estable y no está sobreajustado, lo cual es un buen indicio de que puede generalizar correctamente a datos nuevos.

In [ ]:

# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test_reshaped)
y_pred_classes = y_pred.argmax(axis=1)
y_test_classes = y_test_categorical.argmax(axis=1)

# matriz de confusión
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
conf_matrix_df = pd.DataFrame(conf_matrix, index=activity_labels[1].values, columns=activity_labels[1].values)

fig_confusion = px.imshow(conf_matrix_df, text_auto=True, aspect="auto", title="Matriz de Confusión")
fig_confusion.update_xaxes(title_text="Predicción")
fig_confusion.update_yaxes(title_text="Real")
fig_confusion.show()

# Reporte de clasificación
print("Reporte de Clasificación:")
print(classification_report(y_test_classes, y_pred_classes, target_names=activity_labels[1].values))


93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


Reporte de Clasificación:
                    precision    recall  f1-score   support

           WALKING       0.89      0.96      0.92       532
  WALKING_UPSTAIRS       0.95      0.87      0.91       491
WALKING_DOWNSTAIRS       1.00      1.00      1.00       537
           SITTING       0.90      1.00      0.95       496
          STANDING       0.99      0.89      0.94       420
            LAYING       0.94      0.93      0.93       471

          accuracy                           0.94      2947
         macro avg       0.95      0.94      0.94      2947
      weighted avg       0.94      0.94      0.94      2947



La matriz de confusión muestra el desempeño del modelo en la clasificación de cada actividad en el conjunto de prueba. Cada fila representa las instancias de una clase real, mientras que cada columna representa las predicciones realizadas por el modelo. Observamos que la mayoría de las instancias se encuentran en la diagonal principal, lo que indica predicciones correctas. Sin embargo, hay algunos errores en actividades similares, como "WALKING" y "WALKING_UPSTAIRS", donde el modelo confundió algunas instancias. Este comportamiento es comprensible, ya que estas actividades pueden compartir patrones de movimiento similares.

El reporte de clasificación proporciona métricas de precisión, recall, y F1-score para cada clase, además de un resumen general del modelo. La precisión general es del 94%, lo cual es consistente con los resultados obtenidos anteriormente. Las clases como "WALKING_DOWNSTAIRS" y "SITTING" tienen un F1-score perfecto o cercano a 1.0, lo que indica una excelente capacidad del modelo para clasificarlas. Las métricas de "WALKING" y "WALKING_UPSTAIRS" son ligeramente menores debido a confusiones entre estas actividades. El modelo tiene un desempeño sólido en todas las clases, con un macro promedio y un promedio ponderado de 0.94 en todas las métricas, demostrando una capacidad de clasificación robusta y bien equilibrada.

Explicación
Gráficas de precisión y pérdida: Estas gráficas mostrarán cómo la precisión y la pérdida cambiaron en cada época para el conjunto de entrenamiento y validación, lo cual te ayudará a evaluar el rendimiento y detectar si hay sobreajuste.

Matriz de confusión: La matriz de confusión muestra cómo se desempeñó el modelo en cada actividad, destacando las clases donde el modelo se confundió.

Reporte de clasificación: Esto proporciona métricas detalladas como precisión, exhaustividad (recall) y F1-score para cada clase, lo que te permitirá ver qué actividades se clasificaron mejor o peor.

#### Ajuste de Hiperparámetros para mejoramiento del modelo

A continuación, se realizará un ajuste de hiperparámetros con el objetivo de optimizar aún más el desempeño del modelo. Ajustar hiperparámetros como el número de unidades en las capas LSTM, la tasa de aprendizaje, el tamaño de lote, y el número de épocas permite mejorar la capacidad del modelo para capturar patrones relevantes y reducir el error de predicción. Este proceso es esencial, ya que un modelo bien ajustado no solo incrementa la precisión en el conjunto de prueba, sino que también puede mejorar su capacidad de generalización en datos nuevos. Esto es especialmente importante en aplicaciones como el reconocimiento de actividades humanas, donde los matices en los patrones de movimiento pueden ser sutiles, y una configuración óptima del modelo puede hacer la diferencia en la precisión final.

In [ ]:
# Función para crear el modelo con hiperparámetros
def build_model(hp):
    model = Sequential()
    
    # Primera
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32),
                   input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]),
                   return_sequences=hp.Boolean('second_lstm_layer')))  
    
    # Segunda
    if hp.Boolean('second_lstm_layer'):
        model.add(LSTM(units=hp.Int('units_2', min_value=32, max_value=128, step=32), return_sequences=False))
    
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Capa densa
    model.add(Dense(32, activation='relu'))
    model.add(Dense(y_train_categorical.shape[1], activation='softmax'))
    
    # Compilación del modelo
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:

# ajuste de hiperparámetros
tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='my_dir',
    project_name='tuning_lstm_model'
)


Reloading Tuner from my_dir\tuning_lstm_model\tuner0.json


In [ ]:
tuner.search(X_train_reshaped, y_train_categorical, epochs=20, validation_data=(X_test_reshaped, y_test_categorical), batch_size=32)

# Obtener el mejor modelo y sus hiperparámetros
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Mejores hiperparámetros:")
print(best_hyperparameters.values)


Trial 22 Complete [00h 00m 32s]
val_accuracy: 0.9558873176574707

Best val_accuracy So Far: 0.9626739025115967
Total elapsed time: 00h 06m 05s
Mejores hiperparámetros:
{'units': 96, 'return_sequences': False, 'second_lstm_layer': False, 'dropout': 0.30000000000000004, 'units_2': 32, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}


c:\Users\Alan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 16 variables. 



Se definió una función build_model que permite construir el modelo LSTM con hiperparámetros ajustables. Usando la librería Keras Tuner y el método Hyperband, se realizó una búsqueda exhaustiva de combinaciones de hiperparámetros, tales como el número de unidades en las capas LSTM, si incluir una segunda capa LSTM, la tasa de dropout, y el número de unidades en la capa densa. Luego, se ejecutó el proceso de ajuste (tuner.search) para encontrar los mejores valores de estos hiperparámetros con base en la precisión de validación. Finalmente, se imprimieron los mejores hiperparámetros obtenidos, los cuales lograron una precisión de validación óptima, mejorando así el rendimiento del modelo.

In [ ]:
# Entrenar el mejor modelo
history = best_model.fit(X_train_reshaped, y_train_categorical, epochs=20, batch_size=32, validation_data=(X_test_reshaped, y_test_categorical))

# Evaluar el modelo 
test_loss, test_accuracy = best_model.evaluate(X_test_reshaped, y_test_categorical)
print(f"Precisión en el conjunto de prueba después del ajuste de hiperparámetros: {test_accuracy * 100:.2f}%")


Epoch 1/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9734 - loss: 0.0694 - val_accuracy: 0.9518 - val_loss: 0.1343
Epoch 2/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9763 - loss: 0.0578 - val_accuracy: 0.9542 - val_loss: 0.1370
Epoch 3/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9772 - loss: 0.0557 - val_accuracy: 0.9505 - val_loss: 0.1577
Epoch 4/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9742 - loss: 0.0636 - val_accuracy: 0.9498 - val_loss: 0.1605
Epoch 5/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9805 - loss: 0.0509 - val_accuracy: 0.9080 - val_loss: 0.3295
Epoch 6/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9743 - loss: 0.0695 - val_accuracy: 0.9460 - val_loss: 0.1837
Epoch 7/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9771 - loss: 0.0591 - val_accuracy: 0.9491 - val_loss: 0.1559
Epoch 8/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9804 - loss: 0.0570 - val_accuracy: 0.

Después del ajuste de hiperparámetros en el modelo, obtuvimos una precisión en el conjunto de prueba de 95.08%. A lo largo de las 20 épocas de entrenamiento, observamos una mejora constante tanto en la precisión como en la pérdida del modelo. La precisión de validación se mantuvo alta y estable, oscilando alrededor del 95%, lo cual demuestra que el modelo no presenta problemas de sobreajuste significativo. Este aumento en la precisión frente a la versión inicial del modelo sugiere que el ajuste de hiperparámetros fue exitoso, optimizando el desempeño en el conjunto de datos de prueba y fortaleciendo la capacidad del modelo para clasificar actividades humanas con gran precisión.

In [ ]:

# Gráfica de precisión
fig_accuracy = go.Figure()
fig_accuracy.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines', name='Precisión de Entrenamiento'))
fig_accuracy.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines', name='Precisión de Validación'))
fig_accuracy.update_layout(title='Precisión durante el Entrenamiento (Modelo Ajustado)',
                           xaxis_title='Época', yaxis_title='Precisión')
fig_accuracy.show()

# Gráfica de pérdida
fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Pérdida de Entrenamiento'))
fig_loss.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='Pérdida de Validación'))
fig_loss.update_layout(title='Pérdida durante el Entrenamiento (Modelo Ajustado)',
                       xaxis_title='Época', yaxis_title='Pérdida')
fig_loss.show()


En la primera gráfica de "Precisión durante el Entrenamiento (Modelo Ajustado)", se observa que la precisión de entrenamiento se mantiene consistentemente alta, cerca del 98%, mientras que la precisión de validación experimenta algunas fluctuaciones, especialmente en las primeras épocas, con una caída notable alrededor de la época 4. Sin embargo, después de esta variación inicial, la precisión de validación se estabiliza, manteniéndose en un rango aceptable y cercano al 95%. Este comportamiento sugiere que el modelo es capaz de generalizar bien a pesar de las fluctuaciones iniciales.

En la segunda gráfica de "Pérdida durante el Entrenamiento (Modelo Ajustado)", la pérdida de entrenamiento es consistentemente baja, lo cual es un buen indicador de que el modelo se ajusta adecuadamente a los datos de entrenamiento. La pérdida de validación, por otro lado, muestra picos y caídas en las primeras épocas, similar a la precisión de validación, estabilizándose posteriormente en un nivel aceptable y bajo. Estas variaciones iniciales en la pérdida y precisión de validación pueden deberse al ajuste de hiperparámetros que optimizó el modelo para obtener un mejor desempeño general a medida que avanzaba el entrenamiento.

In [ ]:


# Predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test_reshaped)
y_pred_classes = y_pred.argmax(axis=1)
y_test_classes = y_test_categorical.argmax(axis=1)

#  matriz de confusión
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
activity_labels_list = activity_labels[1].values
conf_matrix_df = pd.DataFrame(conf_matrix, index=activity_labels_list, columns=activity_labels_list)

fig_confusion = px.imshow(conf_matrix_df, text_auto=True, aspect="auto", title="Matriz de Confusión")
fig_confusion.update_xaxes(title_text="Predicción")
fig_confusion.update_yaxes(title_text="Real")
fig_confusion.show()


93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step


La matriz de confusión muestra el desempeño del modelo ajustado en la clasificación de las actividades humanas en el conjunto de prueba. Cada fila representa las instancias reales de cada clase de actividad, mientras que cada columna indica las predicciones realizadas por el modelo para esas actividades. Observamos que el modelo tiene una alta precisión en la mayoría de las actividades, con una correcta clasificación predominante en la diagonal principal, lo cual indica predicciones acertadas.

Sin embargo, hay ciertos errores de clasificación, principalmente en actividades que pueden ser más difíciles de distinguir entre sí, como "WALKING" y "WALKING_UPSTAIRS", donde se observa que algunas instancias de "WALKING_UPSTAIRS" fueron clasificadas como "WALKING". El modelo demuestra un buen rendimiento, aunque existen pequeñas confusiones entre algunas actividades similares, lo cual podría ser una consecuencia de la similitud en los patrones de movimiento entre estas actividades.

In [ ]:


# Graficar la curva ROC para cada clase
fig_roc = go.Figure()
for i, label in enumerate(activity_labels_list):
    fpr, tpr, _ = roc_curve(y_test_categorical[:, i], y_pred[:, i])
    auc_score = auc(fpr, tpr)
    fig_roc.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'{label} (AUC = {auc_score:.2f})'))

fig_roc.update_layout(title='Curvas ROC para cada Clase', xaxis_title='Tasa de Falsos Positivos', yaxis_title='Tasa de Verdaderos Positivos')
fig_roc.show()


La gráfica de curvas ROC para cada clase muestra la relación entre la tasa de verdaderos positivos y la tasa de falsos positivos para cada actividad en el conjunto de prueba. Cada curva representa el desempeño del modelo en la clasificación de una actividad específica, y el área bajo la curva (AUC) indica la capacidad del modelo para distinguir entre las clases. 

Se observa que la mayoría de las clases tienen un AUC cercano a 1.0, lo cual indica un excelente rendimiento del modelo en la separación de las clases. Las actividades como "WALKING_DOWNSTAIRS," "SITTING," "STANDING," y "LAYING" tienen un AUC perfecto de 1.0, lo que sugiere que el modelo tiene una alta precisión en la identificación de estas actividades. Las actividades "WALKING" y "WALKING_UPSTAIRS" tienen un AUC ligeramente inferior (0.99), lo que podría reflejar la dificultad del modelo para distinguir entre estos movimientos similares. Estas curvas ROC demuestran que el modelo es robusto y tiene un excelente desempeño en la clasificación de las actividades humanas.

In [ ]:

fig_pr = go.Figure()
for i, label in enumerate(activity_labels_list):
    precision, recall, _ = precision_recall_curve(y_test_categorical[:, i], y_pred[:, i])
    fig_pr.add_trace(go.Scatter(x=recall, y=precision, mode='lines', name=f'{label}'))

fig_pr.update_layout(title='Curvas de Precisión-Exhaustividad para cada Clase', xaxis_title='Exhaustividad', yaxis_title='Precisión')
fig_pr.show()


La gráfica de curvas de Precisión-Exhaustividad (o Precision-Recall) para cada clase muestra cómo varían la precisión y la exhaustividad del modelo al clasificar cada actividad a medida que se ajusta el umbral de decisión. Cada línea representa el rendimiento del modelo en una clase específica, y la cercanía de las curvas a la esquina superior derecha indica una alta precisión y exhaustividad simultáneamente, lo cual es ideal.

Se observa que la mayoría de las actividades, como "SITTING," "STANDING," "WALKING_DOWNSTAIRS," y "LAYING," presentan un excelente desempeño, manteniendo valores de precisión y exhaustividad cercanos a 1 en la mayoría de los casos. Sin embargo, las actividades como "WALKING" y "WALKING_UPSTAIRS" muestran una ligera disminución en la precisión a medida que la exhaustividad aumenta, lo que sugiere que el modelo podría tener cierta dificultad en distinguir entre actividades de caminata similares. La gráfica indica que el modelo maneja bien la mayoría de las clases con un equilibrio positivo entre precisión y exhaustividad, reforzando la eficacia del modelo en la clasificación de actividades humanas.

#### Conclusión

El desarrollo de este modelo de deep learning para la clasificación de actividades humanas ha demostrado ser altamente efectivo, alcanzando una precisión notable del 95.08% en el conjunto de prueba tras el ajuste de hiperparámetros. Desde la limpieza de datos, en la cual confirmamos la integridad del dataset, hasta la optimización de la arquitectura del modelo mediante técnicas de tuning, cada etapa del proceso ha contribuido a mejorar el desempeño y la robustez de la solución. Las métricas de evaluación, incluidas las curvas ROC y las matrices de confusión, muestran que el modelo puede distinguir con precisión entre actividades similares, aunque todavía persisten pequeños desafíos en la diferenciación de actividades de caminata. Este trabajo no solo valida la aplicabilidad de modelos de LSTM en problemas de clasificación temporal, sino que también destaca la importancia de una optimización cuidadosa y del uso de herramientas visuales para entender el rendimiento del modelo en diferentes clases. En conjunto, el proyecto evidencia cómo la integración de técnicas avanzadas de machine learning puede llevar a resultados sólidos y útiles en el ámbito del reconocimiento de actividades humanas.